In [1]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize, imsave
import scipy.io
from scipy.io import savemat, loadmat
from os import listdir, walk
from os.path import isfile, join
#import PIL
import matplotlib.pyplot as plt
import time

start_time = time.time()

trainPath = 'data/train/'
avgImgHeight = int(360/4)       # Actual average height is 360. I am doing more downsampling
avgImgWidth = int(400/4)        # Actual average width is 400. I am doing more downsampling
imgClasses = ['dog', 'cat']
numClasses = len(imgClasses)

fileNames = next(walk(trainPath))[2]
numTrain = len(fileNames)
trainDataX = np.zeros((numTrain, avgImgHeight, avgImgWidth, 3), dtype=np.uint8)
trainDataY = np.zeros((numTrain, numClasses), dtype=np.uint8)
for i, fileName in enumerate(fileNames):
    img = imread(join(trainPath, fileName))
    img = imresize(img, (avgImgHeight, avgImgWidth))
    trainDataX[i, :, :, :] = img

    for j, imgClass in enumerate(imgClasses):
        trainDataY[i, j] = (fileName[0:3] == imgClass)

print('Time taken to load data = ', time.time()-start_time)

In [6]:
trainDataX = np.reshape(trainDataX, (trainDataX.shape[0], -1))

flatImgDim = trainDataX.shape[1]

learningRate = 0.00000001
numIter = 10000
batchSize = 100
X = tf.placeholder(tf.float32, [None, flatImgDim])
y = tf.placeholder(tf.float32, [None, numClasses])

W = tf.Variable(np.float32(np.random.randn(flatImgDim, numClasses)*0.001))
b = tf.Variable(tf.zeros(numClasses))

y_pred = tf.nn.softmax(tf.matmul(X,W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
trainStep = tf.train.GradientDescentOptimizer(learningRate).minimize(cross_entropy)
correctPrediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

for i in range(numIter):
    sampleIndices = np.random.choice(np.arange(numTrain), batchSize)
    batch_xs = trainDataX[sampleIndices]
    batch_ys = trainDataY[sampleIndices]
    sess.run(trainStep, feed_dict={X: batch_xs, y: batch_ys})
    if i%100 == 0:
        print("Iteration = ", i, "Cross Entropy = ", sess.run(cross_entropy, feed_dict={X:batch_xs, y: batch_ys}),
              "Train Accuracy = ", sess.run(accuracy, feed_dict={X:batch_xs, y: batch_ys}))



print('Training accuracy with Softmax regression = ', sess.run(accuracy, feed_dict={X: trainDataX, y: trainDataY}))

Iteration =  0 Cross Entropy =  4.76406 Train Accuracy =  0.51
Iteration =  100 Cross Entropy =  3.94905 Train Accuracy =  0.52
Iteration =  200 Cross Entropy =  3.9929 Train Accuracy =  0.48
Iteration =  300 Cross Entropy =  3.21758 Train Accuracy =  0.5
Iteration =  400 Cross Entropy =  2.43169 Train Accuracy =  0.56
Iteration =  500 Cross Entropy =  2.98761 Train Accuracy =  0.52
Iteration =  600 Cross Entropy =  3.25279 Train Accuracy =  0.55
Iteration =  700 Cross Entropy =  2.53124 Train Accuracy =  0.57
Iteration =  800 Cross Entropy =  2.10473 Train Accuracy =  0.64
Iteration =  900 Cross Entropy =  3.14365 Train Accuracy =  0.52
Iteration =  1000 Cross Entropy =  2.63763 Train Accuracy =  0.54
Iteration =  1100 Cross Entropy =  3.1464 Train Accuracy =  0.45
Iteration =  1200 Cross Entropy =  3.09896 Train Accuracy =  0.51
Iteration =  1300 Cross Entropy =  2.60916 Train Accuracy =  0.54
Iteration =  1400 Cross Entropy =  2.64653 Train Accuracy =  0.58
Iteration =  1500 Cross E